<div style="text-align:left"><span style="font-size:1.0cm; line-height: 1.2cm; font-family:cursive; ">Google Map 店家資訊爬蟲</span><br></div>

## 引入套件

In [1]:
import os
import re
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup as Soup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

## 搜尋內容
### 建議格式：地區+店家名稱

In [3]:
search = "大安區 星巴克"

## 開啟 Google Map 並查詢

In [ ]:
# 設定工作目錄
os.chdir("/Users/andiology/Downloads/Google Map Review 爬蟲") # 要改
# 設定要前往的網址
url = 'https://www.google.com.tw/maps?hl=zh-TW&tab=rl&authuser=0'  
# 透過 Browser Driver 開啟 Chrome
browser = webdriver.Chrome('./chromedriver')

![](https://imgur.com/zMGv7VD.png)

In [ ]:
# 前往該網址
browser.get(url) 
# 填入搜尋內容
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'tactile-searchbox-input')))
# 網頁元素定位 
search_input = browser.find_elements_by_class_name('tactile-searchbox-input')[0]
# 輸入搜尋內容 
search_input.send_keys(search)

![](https://imgur.com/mDYmifN.png)

In [4]:
# 搜尋
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'searchbox-searchbutton-container')))
# 網頁元素定位 
search_click = browser.find_elements_by_class_name('searchbox-searchbutton-container')[0]
# 點擊搜尋鍵 
search_click.click()
# 成功搜尋
print("Finish Searching!")

Finish Searching!


![](https://imgur.com/1Q7plpV.png)

### 預設產出資料list及時間

In [5]:
i = 1 # 第幾個店家
store_names = []
star_section = []
review_num = []
addresses = []
authorities = []
phones = []
pluscodes = []
all_open_hours = []
today = datetime.today().strftime("%Y-%m-%d")

### While 迴圈
當存在下一頁就 繼續執行

In [ ]:
# Next = True
# while Next:

In [6]:
# 取得網頁元素框架（搜尋結果）
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-result')))
soup = Soup(browser.page_source,"lxml")
store_frame = soup.find_all(class_="section-result")

![](https://imgur.com/aiN8qed.png)

In [7]:
# 找所有店家名稱
store_name = []
for stores in store_frame:
    new_store = stores.get('aria-label')
    if (new_store != None) & (new_store not in store_name):
        store_name.append(new_store)
print("店家名稱：")
for sn in store_name:
    print(sn)

店家名稱：
STARBUCKS 星巴克 (辛亥門市)
STARBUCKS 星巴克 (興和門市)
Starbucks
STARBUCKS 星巴克 (長興門市)
STARBUCKS 星巴克 (敦和門市)
STARBUCKS 星巴克 (敦富門市)
STARBUCKS 星巴克 (通化門市)
STARBUCKS 星巴克 (復安門市)
STARBUCKS 星巴克 (崇興門市)
STARBUCKS 星巴克 (崇光門市)
STARBUCKS 星巴克 (華視門市)
STARBUCKS 星巴克 (同領門市)
STARBUCKS 星巴克 (公館門市)
STARBUCKS 星巴克 (雲和門市)
STARBUCKS 星巴克 (仁愛延吉門市)
STARBUCKS 星巴克 (忠孝延吉門市)
STARBUCKS RESERVE™ 星巴克龍門新概念店


## 進入個別店家

### For 迴圈
執行所有店家

In [8]:
# for n in range(len(store_name)):
n = 0

In [9]:
# 店家名稱
store_names.append(store_name[n])
print('抓抓',store_name[n])
time.sleep(3)

抓抓 STARBUCKS 星巴克 (辛亥門市)


### 點擊進入個別店家 

![](https://imgur.com/102YYFS.png)

In [10]:
# 抓個別店家資料
WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-result')))
store_click = browser.find_elements_by_class_name('section-result')[n]
store_click.click()

![](https://imgur.com/mOiZRyE.png)

In [11]:
# 獲取網頁資訊
soup = Soup(browser.page_source,"lxml")

![](https://imgur.com/wksrbMU.png)

In [12]:
# 店家星級
try:
    star_section.append(float(soup.find_all(class_="section-star-display")[0].text))
except:
    WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-star-display')))
    star_section.append(float(soup.find_all(class_="section-star-display")[0].text))
print("店家星級：", star_section)

店家星級： [4.1]


![](https://imgur.com/wVOpNfB.png)

In [13]:
# 店家總評論數
review_num.append(int(soup.find_all(class_="widget-pane-link")[0].get('aria-label').strip(" 則評論").replace(',','')))
print("店家總評論數：", review_num)

店家總評論數： [990]


### 問題一：
### 要如何爬取店家種類（如：咖啡店）

In [ ]:
# 作答
# 店家種類
store_type = []
store_type_frame = soup.find_all(class_="widget-pane-link")
store_type.append(store_type_frame[-3].text)

print("店家種類：", store_type)
# > 店家種類： ['咖啡店']

![](https://imgur.com/Yy59WST.png)

In [14]:
# 店家地址
address_frame = soup.find_all('button', {'data-item-id':"address"})
addresses.append(address_frame[0].get('aria-label').strip().strip('地址: '))
print("店家地址：", addresses)

店家地址： ['106台北市大安區辛亥路二段175號']


![](https://imgur.com/AKL3FDD.png)

In [15]:
# 店家網站
authority_frame = soup.find_all('button', {'data-item-id':"authority"})
if authority_frame == []:
    authorities.append([])
    print('empty')
else:
    authorities.append(authority_frame[0].get('aria-label').strip().strip('網站: '))
print("店家網站：", authorities)

店家網站： ['starbucks.com.tw']


![](https://imgur.com/iuCdp1h.png)

### 問題二：
### 把 phone_num 中出現的「前後空格」與「'電話號碼: '」去除

In [16]:
# 店家電話號碼
phone_frame = soup.find_all('button', {'data-item-id':re.compile('phone:')})
if phone_frame == []:
    phones.append([])
    print('empty T-T')
else:
    phone_num = phone_frame[0].get('aria-label').strip().strip('電話號碼: ')
    phones.append(phone_num)
print("店家電話號碼：", phones)

店家電話號碼： ['02 2738 5799']


![](https://imgur.com/1u34A8b.png)

In [17]:
# 店家Plus Code
pluscode_frame = soup.find_all('button', {'data-item-id':"oloc"})
pluscodes.append(pluscode_frame[0].div.text.strip())
print("店家Plus Code：", pluscodes)

店家Plus Code： ['2GCR+MF 大安區 台北市']


![](https://imgur.com/4I8hNJj.png)

In [18]:
# 店家營業時間
open_hours_elem = soup.find_all(class_="section-open-hours-container")
if open_hours_elem == []:
    open_hours_sorted = []
else:
    open_hours = open_hours_elem[0].get('aria-label')
    open_hours = open_hours.split('.')[0]
    open_hours_list = open_hours.split('; ')
    weekday = ['星期日','星期一','星期二','星期三','星期四','星期五','星期六']
    open_hours_sorted = []
    for oh in range(len(open_hours_list)):
        for op in open_hours_list:
            if weekday[oh] in op:
                open_hours_sorted.append(op)
all_open_hours.append(open_hours_sorted)
print("店家營業時間：", all_open_hours)

店家營業時間： [['星期日、06:30 到 19:00', '星期一、06:30 到 20:00', '星期二、06:30 到 20:00', '星期三、06:30 到 20:00', '星期四、06:30 到 20:00', '星期五、06:30 到 20:00', '星期六、06:30 到 19:00']]


## 返回搜尋頁面

![](https://imgur.com/ftw5lpv.png)

In [19]:
# 點擊返回搜尋結果
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'section-back-to-list-button.blue-link.noprint')))
back_btn = browser.find_elements_by_class_name('section-back-to-list-button.blue-link.noprint')[0]
back_btn.click()
print('return to search result')

return to search result


### For 迴圈結尾

### 點擊下一頁

![](https://imgur.com/SeBc7Pq.png)

In [20]:
# 點擊下一頁按鈕
try:
    WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, 'n7lv7yjyC35__section-pagination-button-next')))
    next_page = browser.find_elements_by_class_name('n7lv7yjyC35__button.noprint')[1]
    next_page.click()
    # Next = True
    print('Next page!')
    time.sleep(2)
except:
    # Next = False
    print('No next page!')

Next page!


### While 迴圈結尾

### 問題三：
### 將所有店家資訊合併成一個 Data Frame
### 根據以下欄位：
    "Store"
    "Star Rating"
    "Number of Reviews"
    "Store Type"
    "Address"
    "Open Hours"
    "Website"
    "Phone Number"
    "Plus Code"

In [23]:
# 店家資訊合成 Data Frame 
store_info = pd.DataFrame({
    'Store':store_names,
    'Star Rating':star_section,
    'Number of Reviews':review_num,
    'Store Type':store_type,
    'Address':addresses,
    'Open Hours':all_open_hours,
    'Website':authorities,
    'Phone Number':phones,
    'Plus Code':pluscodes,
    })
store_info

,Store,Star Rating,Number of Reviews,Address,Open Hours,Website,Phone Number,Plus Code
0,STARBUCKS 星巴克 (辛亥門市),4.1,990,106台北市大安區辛亥路二段175號,"[星期日、06:30 到 19:00, 星期一、06:30 到 20:00, 星期二、06:...",starbucks.com.tw,02 2738 5799,2GCR+MF 大安區 台北市


In [24]:
# 輸出 csv file
store_info.to_csv('個別店家評論/0_'+ search +'_店家資訊.csv', encoding = "UTF-8-sig")

![](https://imgur.com/hi2wzlt.png)